In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ROOT_PATH = Path().absolute().parent

PosixPath('/home/ingrid/Documents/TDT4900-Master-Thesis')

In [3]:
data_path = ROOT_PATH.joinpath('data')

In [ ]:
# flickr8k

#flick30k

#MSCOCO